### 머신러닝의 지도학습(변수, 정답)의 한 알고리즘 사용
- 나이브 베이즈 분류(Naive Bayes Classifier)
- 두 사건을 서로 독립적이라고 가정, 각각의 조건부 확률을 계산
- 방식
 > 샘플 문장 제시 -> 긍정/부정인지 답안 제시  
 > 학습(문장을 형태소로 분해해서 학습용 데이터로 구성)  
 > 학습 후 모델이 생성되면, 한번도 접하지 않은 문장으로 예측 test

In [1]:
from nltk.tokenize import word_tokenize
import nltk

In [2]:
# 긍정 : pos / 부정 : neg
# 데이터 구성 -> 학습용 : 테스트용 = 75 : 25
train = [ ('i like you', 'pos'), 
         ('i hate you', 'neg'), 
         ('you like me', 'neg'), 
         ('i like her', 'pos') ]

###### word_tokenize()

In [7]:
for sentence in train :
    print(sentence[0])
    # word_tokenize : 한글의 형태소로 분해하는 과정과 유사
    print(word_tokenize(sentence[0]))    # 공백 단위로 문장을 쪼갠다
    for word in word_tokenize(sentence[0]) :
        # 알파벳은 대문자 혹은 소문자로 일치시킨다
        print(word.lower())

i like you
['i', 'like', 'you']
i
like
you
i hate you
['i', 'hate', 'you']
i
hate
you
you like me
['you', 'like', 'me']
you
like
me
i like her
['i', 'like', 'her']
i
like
her


In [12]:
# 전체 문장에서 형태소로 분해하여 중복을 제거하고 리스트로 담아라
# for sentence in train :
#    for word in word_tokenize(sentence[0]) :
#        print(word.lower())
all_words = list(set([word.lower() 
                      for sentence in train 
                      for word in word_tokenize(sentence[0])]))
all_words

['you', 'me', 'her', 'like', 'i', 'hate']

In [16]:
# 훈련용 텍스트 문장에 all_words의 내용을 대비하여 해당 말뭉치가 있는지 없는지 체크
for x in train :
    for word in all_words :
        # print(word)
        print(x[1], { word : word in word_tokenize(x[0]) })

pos {'you': True}
pos {'me': False}
pos {'her': False}
pos {'like': True}
pos {'i': True}
pos {'hate': False}
neg {'you': True}
neg {'me': False}
neg {'her': False}
neg {'like': False}
neg {'i': True}
neg {'hate': True}
neg {'you': True}
neg {'me': True}
neg {'her': False}
neg {'like': True}
neg {'i': False}
neg {'hate': False}
pos {'you': False}
pos {'me': False}
pos {'her': True}
pos {'like': True}
pos {'i': True}
pos {'hate': False}


In [17]:
for x in train :
    print(x[1], { word : word in word_tokenize(x[0]) for word in all_words })

pos {'you': True, 'me': False, 'her': False, 'like': True, 'i': True, 'hate': False}
neg {'you': True, 'me': False, 'her': False, 'like': False, 'i': True, 'hate': True}
neg {'you': True, 'me': True, 'her': False, 'like': True, 'i': False, 'hate': False}
pos {'you': False, 'me': False, 'her': True, 'like': True, 'i': True, 'hate': False}


In [18]:
t = [ ({ word : word in word_tokenize(x[0]) for word in all_words }, x[1])
     for x in train ]
t

[({'you': True,
   'me': False,
   'her': False,
   'like': True,
   'i': True,
   'hate': False},
  'pos'),
 ({'you': True,
   'me': False,
   'her': False,
   'like': False,
   'i': True,
   'hate': True},
  'neg'),
 ({'you': True,
   'me': True,
   'her': False,
   'like': True,
   'i': False,
   'hate': False},
  'neg'),
 ({'you': False,
   'me': False,
   'her': True,
   'like': True,
   'i': True,
   'hate': False},
  'pos')]

In [19]:
# 학습
# 알고리즘 생성 및 학습을 한 번에 진행
classifier = nltk.NaiveBayesClassifier.train(t)

In [20]:
# 학습 결과, 분류의 결과치
classifier.show_most_informative_features()

Most Informative Features
                    hate = False             pos : neg    =      1.7 : 1.0
                       i = True              pos : neg    =      1.7 : 1.0
                    like = True              pos : neg    =      1.7 : 1.0
                      me = False             pos : neg    =      1.7 : 1.0
                     her = False             neg : pos    =      1.7 : 1.0
                     you = True              neg : pos    =      1.7 : 1.0


In [21]:
# test
test_sentence = 'i like MeRui'

In [22]:
# test_sentence를 위처럼 딕셔너리 형태로 구성
test_sentence_feature = { word.lower() : (word in word_tokenize(test_sentence)) for word in all_words }
test_sentence_feature

{'you': False,
 'me': False,
 'her': False,
 'like': True,
 'i': True,
 'hate': False}

In [23]:
# 예측
classifier.classify( test_sentence_feature )

'pos'

#### 한글 데이터

In [24]:
from konlpy.tag import Okt
pos_tagger = Okt()

In [25]:
# 학습 데이터
train = [ ('아웃백이 좋아', 'pos'), 
         ('애슐리도 좋아', 'pos'), 
         ('난 자연당이 싫어', 'neg'), 
         ('아웃백은 아주 좋은 식당이야', 'pos'), 
         ('난 수업 마치고 아웃백에 갈 거야', 'pos') ]

In [28]:
# 말뭉치 준비
# 중복 제거
# nltk의 tokenizer는 공백 기준으로 명사품사 등을 쪼개지 않고 그대로 워드화

# for sentence in train :
#     for word in word_tokenize(sentence[0]) :
#         print(word)
all_words = set([word 
                 for sentence in train 
                 for word in word_tokenize(sentence[0])])
all_words

{'갈',
 '거야',
 '난',
 '마치고',
 '수업',
 '식당이야',
 '싫어',
 '아웃백에',
 '아웃백은',
 '아웃백이',
 '아주',
 '애슐리도',
 '자연당이',
 '좋아',
 '좋은'}

In [29]:
t = [ ({ word : (word in word_tokenize(x[0])) for word in all_words }, x[1]) 
        for x in train ]
t

[({'거야': False,
   '식당이야': False,
   '좋아': True,
   '수업': False,
   '갈': False,
   '자연당이': False,
   '아주': False,
   '싫어': False,
   '애슐리도': False,
   '난': False,
   '아웃백은': False,
   '마치고': False,
   '아웃백에': False,
   '아웃백이': True,
   '좋은': False},
  'pos'),
 ({'거야': False,
   '식당이야': False,
   '좋아': True,
   '수업': False,
   '갈': False,
   '자연당이': False,
   '아주': False,
   '싫어': False,
   '애슐리도': True,
   '난': False,
   '아웃백은': False,
   '마치고': False,
   '아웃백에': False,
   '아웃백이': False,
   '좋은': False},
  'pos'),
 ({'거야': False,
   '식당이야': False,
   '좋아': False,
   '수업': False,
   '갈': False,
   '자연당이': True,
   '아주': False,
   '싫어': True,
   '애슐리도': False,
   '난': True,
   '아웃백은': False,
   '마치고': False,
   '아웃백에': False,
   '아웃백이': False,
   '좋은': False},
  'neg'),
 ({'거야': False,
   '식당이야': True,
   '좋아': False,
   '수업': False,
   '갈': False,
   '자연당이': False,
   '아주': True,
   '싫어': False,
   '애슐리도': False,
   '난': False,
   '아웃백은': True,
   '마치고': False,
   '아웃백에': False,
   '아웃백

In [30]:
classifier = nltk.NaiveBayesClassifier.train(t)

In [31]:
classifier.show_most_informative_features()

Most Informative Features
                       난 = True              neg : pos    =      2.5 : 1.0
                      좋아 = False             neg : pos    =      1.5 : 1.0
                      좋은 = False             neg : pos    =      1.1 : 1.0
                    아웃백이 = False             neg : pos    =      1.1 : 1.0
                     마치고 = False             neg : pos    =      1.1 : 1.0
                       갈 = False             neg : pos    =      1.1 : 1.0
                    아웃백에 = False             neg : pos    =      1.1 : 1.0
                      아주 = False             neg : pos    =      1.1 : 1.0
                    애슐리도 = False             neg : pos    =      1.1 : 1.0
                    식당이야 = False             neg : pos    =      1.1 : 1.0


In [32]:
# 예측할 문장
test_sentence = '난 수업 마치고 아웃백에 갈 거야'

In [34]:
test_sentence_feature = { word : (word in word_tokenize(test_sentence)) for word in all_words }
test_sentence_feature

{'거야': True,
 '식당이야': False,
 '좋아': False,
 '수업': True,
 '갈': True,
 '자연당이': False,
 '아주': False,
 '싫어': False,
 '애슐리도': False,
 '난': True,
 '아웃백은': False,
 '마치고': True,
 '아웃백에': True,
 '아웃백이': False,
 '좋은': False}

In [36]:
classifier.classify(test_sentence_feature)

'pos'

In [ ]:
# 위 방식을 진행을 하려면 훈련에 필요한 다량의 데이터(문장, 상황)이 필요하다
# 훈련량이 적거나, 데이터가 적으면 부정확하다(정확도가 떨어진다)
# 절차적인 것만 체크, 한계적인 상황을 고려
# 지도학습은 정답을 알고 있어야 한다.
# 다양하게 구성할 수 있는 문장에 대한 판단이 쉽지 않다
# => 많은 문장과 문서 중에서 유사한 문장을 찾아내고
# => 문장을 벡터로 표현하여, 벡터 간의 거리를 구하여 해결

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
vectorizer = CountVectorizer(min_df = 1) # 1은 임시값

In [40]:
# 많은 문장을 훈련시커야 한다
# 여기서는 몇 가지 샘플로 진행(절차에 집중)
contents = [
    '길동이랑 술마시고 싶지만 바쁜데 어떡하죠?', 
    '길동이는 공원에서 산책하고 노는 것을 싫어해요', 
    '길동이는 공원에서 노는 것도 싫어해요. 이상해요.', 
    '먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요.'
]

In [41]:
X = vectorizer.fit_transform(contents)

In [42]:
vectorizer.get_feature_names()

['것도',
 '것을',
 '곳으로',
 '공원에서',
 '그러질',
 '길동이는',
 '길동이랑',
 '너무',
 '노는',
 '떠나고',
 '바빠서',
 '바쁜데',
 '산책하고',
 '술마시고',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떡하죠',
 '여행을',
 '이상해요',
 '있어요',
 '하고']

In [45]:
# 각 feature에 대한 벡터값
trans = X.toarray().transpose()
trans.shape, trans
# 해석
# '것도'라는 feature는
# 훈련용 전체 문장 4개 중 3번째에 등장한다
# -> [0, 0, 1, 0]이라는 벡터값을 가지게 된다

((22, 4), array([[0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1]], dtype=int64))

In [46]:
X.shape

(4, 22)

In [47]:
num_samples, num_features = X.shape

In [48]:
# 문장이 4개
num_samples

4

In [49]:
# 문장 4개에서 추출한 말뭉치 22개
num_features

22

In [50]:
# 새로운 문장
new_post = ['길동이랑 공원에서 산책하고 놀고 싶어요']

In [51]:
new_post_vec = vectorizer.transform(new_post)

In [54]:
# 문장의 벡터화
new_post_vec.toarray()

array([[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

In [55]:
# 유사도 판단을 하기 위해서 거리 계산
import scipy as sp

In [56]:
# 거리 계산
def dis_raw(v1, v2) :
    delta = v1-v2
    return sp.linalg.norm(delta.toarray())

In [59]:
# 샘플 문장을 반복하면서 이 문장과 일일이 비교
# -> 그 중에서 거리가 가장 짧은 문장 선택
# 그 문장이 가장 유사한 문장이다
best_distance = 1000 # 임의값
best_index = None
for i in range(num_samples) :
    # 각 문장의 벡터 정보를 return
    post_vec = X.getrow(i)
    # 비교 문장 : new_post_vec
    d = dis_raw(post_vec, new_post_vec)
    print("%d %f" % (i, d))
    if d < best_distance :
        best_distance = d
        best_index = i

0 2.449490
1 2.236068
2 2.645751
3 3.464102


In [60]:
# 최소 거리와 그 때의 인덱스
best_distance, best_index

(2.23606797749979, 1)

In [61]:
# 그 때 문장은
contents[best_index]

'길동이는 공원에서 산책하고 노는 것을 싫어해요'

In [ ]:
# new_post = ['길동이랑 공원에서 산책하고 놀고 싶어요']와
# 가장 유사도를 가진다

In [ ]:
# -> nltk의 말뭉치는 한글과 딱히 맞지 않다
# 한글 전용 형태소 분석기를 활용하여 보다 정확하고 의미있게 작업을 진행